In [1]:
import pandas as pd
df = pd.read_csv('../../data/df_clean.csv')

In [2]:
df.head()

,date,eurusd,jpyusd,gbpusd,nq,sp500,nikkei,ftse,gdp_us,gdp_uk,...,unrate_uk,unrate_jp,unrate_ez,m0_us,m0_jp,m0_ez,ir_us,ir_uk,ir_ez,ir_jp
0,2004-01-01,1.258194,0.009320,1.785204,1468.0,1111.920044,10676.639648,4476.899902,11923447.0,322790.0,...,4.8,5.1,9.4,736400000.0,1.060327e+09,548711.0,1.0,3.75,2.0,0.1
1,2004-01-02,1.258194,0.009354,1.793690,1466.5,1108.479980,10676.639648,4510.200195,11923447.0,322790.0,...,4.8,5.1,9.4,736400000.0,1.060327e+09,548711.0,1.0,3.75,2.0,0.1
2,2004-01-05,1.268698,0.009413,1.807207,1495.5,1122.219971,10825.169922,4513.299805,11923447.0,322790.0,...,4.8,5.1,9.4,736400000.0,1.060327e+09,548711.0,1.0,3.75,2.0,0.1
3,2004-01-06,1.272103,0.009414,1.821295,1504.5,1123.670044,10813.990234,4505.200195,11923447.0,322790.0,...,4.8,5.1,9.4,736400000.0,1.060327e+09,548711.0,1.0,3.75,2.0,0.1
4,2004-01-07,1.264095,0.009418,1.817785,1517.0,1126.329956,10757.820312,4473.000000,11923447.0,322790.0,...,4.8,5.1,9.4,736400000.0,1.060327e+09,548711.0,1.0,3.75,2.0,0.1


In [3]:
# Calcular la variación (proporción) para cada tipo de cambio
df['eurusd_pct_change'] = df['eurusd'].pct_change()
df['jpyusd_pct_change'] = df['jpyusd'].pct_change()
df['gbpusd_pct_change'] = df['gbpusd'].pct_change()

# Crear las etiquetas de si sube o baja en inglés
df['eurusd_trend'] = df['eurusd_pct_change'].apply(lambda x: 'up' if x > 0 else 'down')
df['jpyusd_trend'] = df['jpyusd_pct_change'].apply(lambda x: 'up' if x > 0 else 'down')
df['gbpusd_trend'] = df['gbpusd_pct_change'].apply(lambda x: 'up' if x > 0 else 'down')

# Mostrar las primeras filas del DataFrame modificado para verificar los cambios
df.head()


,date,eurusd,jpyusd,gbpusd,nq,sp500,nikkei,ftse,gdp_us,gdp_uk,...,ir_us,ir_uk,ir_ez,ir_jp,eurusd_pct_change,jpyusd_pct_change,gbpusd_pct_change,eurusd_trend,jpyusd_trend,gbpusd_trend
0,2004-01-01,1.258194,0.009320,1.785204,1468.0,1111.920044,10676.639648,4476.899902,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,NaN,NaN,NaN,down,down,down
1,2004-01-02,1.258194,0.009354,1.793690,1466.5,1108.479980,10676.639648,4510.200195,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,0.000000,0.003648,0.004753,down,up,up
2,2004-01-05,1.268698,0.009413,1.807207,1495.5,1122.219971,10825.169922,4513.299805,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,0.008348,0.006307,0.007536,up,up,up
3,2004-01-06,1.272103,0.009414,1.821295,1504.5,1123.670044,10813.990234,4505.200195,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,0.002684,0.000094,0.007795,up,up,up
4,2004-01-07,1.264095,0.009418,1.817785,1517.0,1126.329956,10757.820312,4473.000000,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,-0.006295,0.000471,-0.001927,down,up,down


In [4]:
import yfinance as yf
from datetime import datetime, timedelta

# Obtener la fecha del primer día en el DataFrame
first_date_str = df.loc[0, 'date']
first_date = datetime.strptime(first_date_str, '%Y-%m-%d')

# Descargar los precios de cierre del día anterior para EUR/USD, JPY/USD y GBP/USD
tickers = ['EURUSD=X', 'JPYUSD=X', 'GBPUSD=X']
previous_date = first_date - timedelta(days=1)

# Descargar los datos usando yfinance
data = yf.download(tickers, start=previous_date.strftime('%Y-%m-%d'), end=first_date.strftime('%Y-%m-%d'))

# Obtener el precio de cierre del día anterior
previous_close = data['Close'].iloc[0]

# Calcular la variación porcentual para cada tipo de cambio
df['eurusd_pct_change'] = df['eurusd'].pct_change()
df['jpyusd_pct_change'] = df['jpyusd'].pct_change()
df['gbpusd_pct_change'] = df['gbpusd'].pct_change()

# Rellenar la variación porcentual para la primera observación
df.loc[0, 'eurusd_pct_change'] = (df.loc[0, 'eurusd'] - previous_close['EURUSD=X']) / previous_close['EURUSD=X']
df.loc[0, 'jpyusd_pct_change'] = (df.loc[0, 'jpyusd'] - previous_close['JPYUSD=X']) / previous_close['JPYUSD=X']
df.loc[0, 'gbpusd_pct_change'] = (df.loc[0, 'gbpusd'] - previous_close['GBPUSD=X']) / previous_close['GBPUSD=X']

# Crear las etiquetas de tendencia en inglés
df['eurusd_trend'] = df['eurusd_pct_change'].apply(lambda x: 'up' if x > 0 else 'down')
df['jpyusd_trend'] = df['jpyusd_pct_change'].apply(lambda x: 'up' if x > 0 else 'down')
df['gbpusd_trend'] = df['gbpusd_pct_change'].apply(lambda x: 'up' if x > 0 else 'down')

# Mostrar las primeras filas del DataFrame modificado para verificar los cambios
df.head()


[*********************100%%**********************]  3 of 3 completed


,date,eurusd,jpyusd,gbpusd,nq,sp500,nikkei,ftse,gdp_us,gdp_uk,...,ir_us,ir_uk,ir_ez,ir_jp,eurusd_pct_change,jpyusd_pct_change,gbpusd_pct_change,eurusd_trend,jpyusd_trend,gbpusd_trend
0,2004-01-01,1.258194,0.009320,1.785204,1468.0,1111.920044,10676.639648,4476.899902,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,-0.000642,0.000932,-0.002017,down,up,down
1,2004-01-02,1.258194,0.009354,1.793690,1466.5,1108.479980,10676.639648,4510.200195,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,0.000000,0.003648,0.004753,down,up,up
2,2004-01-05,1.268698,0.009413,1.807207,1495.5,1122.219971,10825.169922,4513.299805,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,0.008348,0.006307,0.007536,up,up,up
3,2004-01-06,1.272103,0.009414,1.821295,1504.5,1123.670044,10813.990234,4505.200195,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,0.002684,0.000094,0.007795,up,up,up
4,2004-01-07,1.264095,0.009418,1.817785,1517.0,1126.329956,10757.820312,4473.000000,11923447.0,322790.0,...,1.0,3.75,2.0,0.1,-0.006295,0.000471,-0.001927,down,up,down


In [5]:
df.to_csv('../../data/df_final.csv', index=False)